Nhiệm vụ 2 – Truy vấn & thống kê mô tả: 
Mục tiêu : thực hành truy xuất dữ liệu và rút ra các thống kê cơ bản. 
- Thống kê số bệnh nhân theo khoa.
- Tìm các lượt khám không có chẩn đoán.
- Thống kê tuổi trung bình theo khoa.
- Tìm bệnh nhân có nhiều lượt khám nhất.
- Tính tỷ lệ phủ BHYT theo lượt khám.

Đọc dữ liệu từ file đã làm sạch


In [2]:
import pandas as pd

#đọc dữ liệu từ file làm sạch
base_path = r"D:\OneDrive\Máy tính\18A2_Nhom_4_Case_9\data_clean"

# Đọc 3 file csv
df_patient = pd.read_csv(f"{base_path}\\patient_clean.csv")
df_visit = pd.read_csv(f"{base_path}\\visit_clean.csv")
df_bill = pd.read_csv(f"{base_path}\\billing_clean.csv")

# 2. KẾT NỐI DỮ LIỆU (
# Nối Visit với Patient qua 'patient_id' để có: Tuổi + Khoa
df = df_visit.merge(df_patient, on='patient_id', how='left')

# Nối tiếp với Billing qua 'visit_id' để có: Thông tin BHYT
df = df.merge(df_bill, on='visit_id', how='left')

print(f"Đã gộp dữ liệu thành công! Tổng số dòng: {len(df)}")

Đã gộp dữ liệu thành công! Tổng số dòng: 164


Thống kê số bệnh nhân theo khoa.

In [3]:
print("\n--- 1. Thống kê số bệnh nhân theo khoa ---")
# Đếm mã bệnh nhân không trùng lặp (nunique)
print(df.groupby('department')['patient_id'].nunique())


--- 1. Thống kê số bệnh nhân theo khoa ---
department
Ngoai    19
Nhi      25
Noi      22
San      23
Name: patient_id, dtype: int64


Tìm các lượt khám không có chẩn đoán

In [4]:
print("\n--- 2. Tìm các lượt khám không có chẩn đoán ---")
# Lọc: diagnosis là NaN hoặc chuỗi rỗng
filter_no_diag = df['diagnosis'].isna() | (df['diagnosis'] == '')
print(df.loc[filter_no_diag, ['visit_id', 'department', 'diagnosis']].head())


--- 2. Tìm các lượt khám không có chẩn đoán ---
  visit_id department diagnosis
3   VK2003      Ngoai       NaN
5   VK2005      Ngoai       NaN
6   VK2006        Nhi       NaN
7   VK2007      Ngoai       NaN
9   VK2009        Noi       NaN


- Thống kê tuổi trung bình theo khoa.

In [5]:
print("\n--- 3. Thống kê tuổi trung bình theo khoa ---")
print(df.groupby('department')['age'].mean())


--- 3. Thống kê tuổi trung bình theo khoa ---
department
Ngoai    31.500000
Nhi      32.156863
Noi      31.452381
San      33.883721
Name: age, dtype: float64


- Tìm bệnh nhân có nhiều lượt khám nhất

In [6]:
print("\n--- 4. Tìm bệnh nhân có nhiều lượt khám nhất ---")
# Đếm tần suất xuất hiện của patient_id
top_bn = df['patient_id'].value_counts()
print(f"Bệnh nhân: {top_bn.idxmax()} - Số lượt: {top_bn.max()}")


--- 4. Tìm bệnh nhân có nhiều lượt khám nhất ---
Bệnh nhân: BN139 - Số lượt: 9


- Tính tỷ lệ phủ BHYT theo lượt khám.

In [35]:

    # Nối Visit với Patient
df = df_visit.merge(df_patient, on='patient_id', how='left')
    # Nối tiếp với Billing (chứa thông tin BHYT)
df = df.merge(df_bill, on='visit_id', how='left') 
    # Xóa khoảng trắng và ký tự %
df['bhyt_clean'] = df['insurance_coverage'].astype(str).str.strip()
df['bhyt_clean'] = df['bhyt_clean'].str.replace('%', '')
    # Chuyển đổi sang kiểu số (lỗi/NaN sẽ thành NaN)
df['bhyt_clean'] = pd.to_numeric(df['bhyt_clean'], errors='coerce')
    # Xử lý logic: Chia 100 nếu giá trị lớn hơn 1 (ví dụ: 80 -> 0.8)
df.loc[df['bhyt_clean'] > 1, 'bhyt_clean'] = df['bhyt_clean'] / 100
    # Xử lý logic: Chuyển giá trị âm thành NaN (sử dụng float('nan'))
df.loc[df['bhyt_clean'] < 0, 'bhyt_clean'] = float('nan')
    # --- 3. TÍNH TOÁN TỶ LỆ PHỦ BHYT CHUNG THEO LƯỢT KHÁM ---
    # Lọc ra danh sách chỉ chứa các dòng có thông tin BHYT hợp lệ (không phải NaN)
df_valid = df.dropna(subset=['bhyt_clean']).copy()
    # Tạo cột đánh dấu BHYT: 1 nếu bhyt_clean > 0, 0 nếu bhyt_clean = 0
df_valid['Has_BHYT'] = (df_valid['bhyt_clean'] > 0).astype(int)
    #Tỷ lệ lượt khám có BHYT (là giá trị trung bình của cột 1/0)
ty_le_co_bhyt = df_valid['Has_BHYT'].mean() * 100
    # Mức hưởng BHYT trung bình
muc_huong_tb = df_valid['bhyt_clean'].mean() * 100
    # Tổng số lượt hợp lệ
tong_so_luot_hop_le = df_valid.shape[0]
    # --- 4. IN KẾT QUẢ ---
print("--- KẾT QUẢ TỶ LỆ PHỦ BHYT THEO LƯỢT KHÁM ---")
print(f"Đã gộp dữ liệu thành công! Tổng số dòng sau khi gộp: {len(df)}")
print(f"Tổng số lượt khám hợp lệ để tính toán: {tong_so_luot_hop_le}")
print(f"Mức hưởng BHYT trung bình: {muc_huong_tb:.2f}%")
print(f"Tỷ lệ lượt khám có BHYT: {ty_le_co_bhyt:.2f}%")



--- KẾT QUẢ TỶ LỆ PHỦ BHYT THEO LƯỢT KHÁM ---
Đã gộp dữ liệu thành công! Tổng số dòng sau khi gộp: 164
Tổng số lượt khám hợp lệ để tính toán: 99
Mức hưởng BHYT trung bình: 55.15%
Tỷ lệ lượt khám có BHYT: 100.00%
